# Bitcoin Price Prediction

##### Feito por: Henrique, Omar, Nevton

### Introdução
#### As criptomoedas estão cada vez mais ganhando espaço no mercado financeiro. Muitos dizem que as criptomoedas não passam de uma bolha especulativa, porém mesmo após uma forte queda a partir de 2018, o preço desses ativos em geral se mantem em um patamar relevante. Devido a alta volatilidade achamos que seria interessante fazer uma análise preditiva sobre o preço dos ativos uma vez que com grande volartilidade existem maiores oportunidades para se ganhar dinheiro. Escolhemos fazer apenas com Bitcoin, pois é a primeira, maior e principal criptomoeda.  
#### Dessa forma nosso objetivo era criar um modelo para tentar prever o preço do bitcoin, para a partir disso tomar decisões de investimento. Para fazer isso, primeiramente coletamos diversas variáveis sobre a rede do Bitcoin, nos ultimos 5 anos. Essas informações podem ser achadas no link: https://www.blockchain.com/charts. E relacionamos essas variáveis com nossa variável chave que era market_price2 (preço em dólares)
#### Nossa pergunta motriz foi: Qual vai ser o preço do Bitcoin No Futuro?

#### Para responder essa pergunta fizemos um regressão multipla e também usamos um método chamado Decision Tree Regressor

#### O que é regressão múltipla?  Explicar
#### O que é Decision Tree Regressor

In [ ]:
import numpy as np
import pandas as pd
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

In [ ]:
dfs = pd.read_excel('DataFrame_bitcoin.xlsx')

### Análise exploratória
Primeiramente foi feito uma análise exploratória dos dados. Dados que estamos cruzando variáveis quantitativas o melhor método para vizualizarmos é com um scatter plot. Por termos um número grande de variáveis, primeiro fizemos uma tabela de correlação e analisamos, através do scatter plot, apenas as variáveis que mais se correlacionavam com nossa variável de interrese: 'Market_price_2'

In [ ]:
tabela_corr = dfs.corr()
tabela_corr

In [ ]:
def correlation_heatmap(df, title, absolute_bounds=True):
    '''Plot a correlation heatmap for the entire dataframe'''
    heatmap = go.Heatmap(
        z=df.corr(method='pearson').values,
        x=df.columns,
        y=df.columns,
        colorbar=dict(title='Pearson Coefficient'),
    )
    
    layout = go.Layout(title=title, width= 800, height = 700)
    
    if absolute_bounds:
        heatmap['zmax'] = 1.0
        heatmap['zmin'] = -1.0
        
    fig = go.Figure(data=[heatmap], layout=layout)
    py.iplot(fig)

In [ ]:
correlation_heatmap(dfs, 'Correlação das variávies')

### Como podemos ver as variáveis que apresentam a correlação mais forte com o market price (preço) são:  
-Custo Por Transação Em Porcentagem  (miners revenue as percentage of the transaction volume.)<br>
-Dificuldade  
-Hash_rate<br>
-Market_Cap<br>
-Volume Estimado Da Transacao<br>
#### Agora vamos analisar mais a fundo essas relações

In [ ]:
plt.scatter(dfs["market_price_2"],dfs["cost_per_transaction_percent"]/10000)
plt.title('Preço x Custo percentual de transação')
plt.xlabel('Preço')
plt.ylabel('Custo percentual de transação')

Notasse que não temos muitos outliers e que essas variáveis tem uma relacão linear positiva

In [ ]:
plt.scatter(dfs["market_price_2"],dfs["difficulty"])
plt.title('Preço x Dificulade')
plt.xlabel('Preço')
plt.ylabel('Dificuladade')

Podemos ver que quanto mais difícil é para minerar um bloco maior o preço

In [ ]:
plt.scatter(dfs["market_price_2"],dfs["hash_rate"])
plt.title('Preço x Hash Rate')
plt.xlabel('Preço')
plt.ylabel('Hash Rate')

A relação é muito parecida com a descrita a cima. Ao fazer uma análise mais profunda sobre os conceitos intrincecos dessas duas variáveis, percebe-se que no fundo o hash rate e a dificuldade são quase a mesma coisa.  
Pode-se encontrar mais informações sobre esses conceitos em : https://2miners.com/blog/mining-difficulty-and-network-hashrate-explained/

In [ ]:
plt.scatter(dfs["market_price_2"],dfs["market_cap"])
plt.title('Preço x Market_Cap')
plt.xlabel('Preço')
plt.ylabel('Market_Cap')

O market cap se mostrou muito relacionado com o preço, porém ao analisar o que a variável significa, concluimos que o market cap é um desdobramento do preço. Formula para se achar o market cap = Preço * quantidade de moedas em circulação

In [ ]:
plt.scatter(dfs["market_price_2"],dfs["estimated_transaction_volume_usd"])
plt.title('Preço x Volumne Estimado Da Transacao')
plt.xlabel('Preço')
plt.ylabel('Volumne Estimado Da Transacao')

Podemos observar uma correlacão forte positiva, ou seja, quanto maior o volume de transações maior o preço. Isso nos indica que se a rede esta mais ativa, mais pessoas querendo transacionar, o preco tende a aumentar

In [ ]:
df_treinamento = dfs.iloc[550:991]
df_teste = dfs.iloc[991:]

In [ ]:
Y_treinamento = df_treinamento['market_price_2']
X_treinamento = df_treinamento[[
 'avg_block_size',
 'cost_per_transaction',
 'difficulty',
 'estimated_transaction_volume_usd',
 'hash_rate',
 'miners_revenue',
 'n_transactions',
 'transaction_fees'
]]
Y_teste = df_teste['market_price_2']
X_teste = df_teste[[
 'avg_block_size',
 'cost_per_transaction',
 'difficulty',
 'estimated_transaction_volume_usd',
 'hash_rate',
 'miners_revenue',
 'n_transactions',
 'transaction_fees'
]]

## Fazendo a Regressão linear multipla

In [ ]:
def regress(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    return results

In [ ]:
regress(X_treinamento,Y_treinamento).summary()

#### Em nossa primeira regressão obtivemos um r-squared de 75.6%, ou seja, conseguimos explicar o modelo em 75.6%

#### Neste primeiro modelo, no entanto estamos apenas usando os dados de um dia atrás, o que torna nosso modelo muito simples e falho. Dessa forma vamos varias a janela temporal usada para verificar qual a melhor janela de tempo para se usar

In [ ]:
n = len(Y_treinamento)
lista = []
dias = []
for i in range(20):
    Y_offset = Y_treinamento.values[i:]
    X_offset = X_treinamento.values[:(n-i),:]
    results = regress(X_offset,Y_offset)
    lista.append(results)
    dias.append(i)
    print(i, results.rsquared)

#### Decidimos fazer a analise usando 10 períodos anteriores, uma vez que analisando até 200 períodos esse foi com maior precisão, ainda sim usando dados anteriores

In [ ]:
i = 10
Y_offset1 = Y_treinamento.values[i:]
X_offset1 = X_treinamento.values[:(n-i),:]
modelo = regress(X_offset1,Y_offset1)
modelo.summary()

In [ ]:
X_test_c = sm.add_constant(X_teste)

In [ ]:
X_test_c.head()

In [ ]:
y_pred=modelo.predict(X_test_c)

In [ ]:
r2_score(Y_teste, y_pred)

Como podemos analisar no resultado da regressão tivemos algumas variáveis com um  valor p muito alto. Agora vamos retirar essas variáveis para ver se conseguimos aumentar a precisão

In [ ]:
X_treinamento1 = df_treinamento[[
 'avg_block_size',
 'cost_per_transaction',
 'n_transactions',
 'transaction_fees'
]]
X_teste1 = df_teste[[
 'avg_block_size',
 'cost_per_transaction',
 'n_transactions',
 'transaction_fees'
]]

In [ ]:
for i in range(200):
    if i == 10:
        Y_offset2 = Y_treinamento.values[i:]
        X_offset2 = X_treinamento1.values[:(n-i),:]
        results2 = regress(X_offset2,Y_offset2)
results2.summary()

Ao tirar as variáveis é possível perceber que não houve melhora na performace do código

#### Como podemos observar atingimos um r-squared de 90.6%, ou seja conseguimos aprimorar nosso modelo inicial em quase 15%, uma melhora significativa. Ainda assim temos 10% de defasagem na análise e em um cenário de investimento essa acurácia ainda nos deixaria muito expostos ao risco da impressão do modelo

## Fazendo o  Decision Tree Regressor

In [ ]:
#Decision Tree Regressor
Arvore = DecisionTreeRegressor(random_state = 1)
Arvore.fit(X_treinamento, Y_treinamento)
plt.scatter(Y_teste, Arvore.predict(X_teste), color = 'blue')
plt.show()

In [ ]:
y_pred1=Arvore.predict(X_teste)
r2_score(Y_teste, y_pred1)

In [ ]:
plt.plot(Y_treinamento)

## Fazendo o Random Forest Regressor

In [ ]:
RandomF = RandomForestRegressor(random_state=0)
RandomF.fit(X_treinamento1, Y_treinamento)
plt.scatter(Y_teste, RandomF.predict(X_teste1), color = 'blue')
plt.show()

In [ ]:
y_pred2=RandomF.predict(X_teste1)
r2_score(Y_teste, y_pred2)